In [1]:
from googleapiclient.discovery import build
import os

In [2]:
from keys import api_key, api_key2

In [3]:
from datetime import datetime, timedelta, timezone


In [4]:
from datetime import datetime, timedelta
import pytz

In [5]:
import pytz

In [6]:
youtube = build('youtube', 'v3', developerKey=api_key2)

In [7]:
def get_latest_video_id(channel_id):
    # Retrieve the list of videos uploaded to the channel.
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        maxResults=7,
        order="date",  # Order by date to get the latest video
        type="video"   # Look for videos only (no playlists or channels)
    )
    response = request.execute()

    for item in response.get('items', []):
        if item['snippet']['liveBroadcastContent'] == 'none':
            video_id = item['id']['videoId']
            video_published_at = datetime.strptime(item['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
            video_published_at = video_published_at.replace(tzinfo=pytz.UTC)
            now = datetime.now(pytz.UTC)

            # Check if the video is not older than 2 days
            if now - video_published_at <= timedelta(days=1):
                return video_id

def get_latest_video_ids(video_ids, channel_ids):
    # Loop over channel IDs and append the video IDs to the list
    for channel_id in channel_ids:
        video_id = get_latest_video_id(channel_id)
        if video_id:
            video_ids.append(video_id)
            print(f"Added Video ID: {video_id}")

# Example usage:
channel_ids = ['UCvJZEG5x-DVYZKTz--pS39w', 'UC7QHSNOKZvbqpXWFv0SiYDQ',"UCinBfsRWiOlzxg7DvjZ5mdQ", "UCUP_ao_7-Yct5FcVIA4Kobg", "UCAHJqpJPqqLLi7siVQpD-3A" , 'UCfdPOTevbfCh_QHsyPeZ8MQ',
               "UCYKtr6GfycBqQJf32tbQSbQ", 'UCpBPTKot6Im9SWs1upQMZKA', 'UCTeFsS-bP0XEt3NBMjfW2cA', 'UC-m6zNItyoDk5lSykDlhE4Q' , 'UCmlfS56nZsnX_5F1fjvX0QA', 'UCoAuxtwgh8LSHeMFqEwjgHA']  # List of example channel IDs
latest_video_ids = []
get_latest_video_ids(latest_video_ids, channel_ids)

print("Latest video IDs:", latest_video_ids)

Added Video ID: 2N8nKIEQU7M
Added Video ID: n_nphlW8xLA
Added Video ID: QsHFTQ-F62Q
Added Video ID: pko04FA7Qvw
Added Video ID: -Ofa8tkQpc8
Added Video ID: lFPcCmiH60o
Added Video ID: p0fr1_Dr37Q
Added Video ID: jo-wA9JkScQ
Added Video ID: -2w2zMdBV3k
Added Video ID: x99XqluD_d0
Latest video IDs: ['2N8nKIEQU7M', 'n_nphlW8xLA', 'QsHFTQ-F62Q', 'pko04FA7Qvw', '-Ofa8tkQpc8', 'lFPcCmiH60o', 'p0fr1_Dr37Q', 'jo-wA9JkScQ', '-2w2zMdBV3k', 'x99XqluD_d0']


In [8]:
latest_video_ids

['2N8nKIEQU7M',
 'n_nphlW8xLA',
 'QsHFTQ-F62Q',
 'pko04FA7Qvw',
 '-Ofa8tkQpc8',
 'lFPcCmiH60o',
 'p0fr1_Dr37Q',
 'jo-wA9JkScQ',
 '-2w2zMdBV3k',
 'x99XqluD_d0']

In [9]:
latest_video_ids = [get_latest_video_id(channel_id) for channel_id in channel_ids if get_latest_video_id(channel_id) is not None]
print(latest_video_ids)

['2N8nKIEQU7M', 'n_nphlW8xLA', 'QsHFTQ-F62Q', 'pko04FA7Qvw', '-Ofa8tkQpc8', 'lFPcCmiH60o', 'p0fr1_Dr37Q', 'jo-wA9JkScQ', '_mp2WOJI11I', 'x99XqluD_d0']


In [10]:
['Tx-WtFiGgKw', 'go-rbLUxtdA', '4fhLDOPiacE', 'HR58Svuetw8', '0iTb19qV4Kw', 'XSBaSS6HEOw', 'NH6S8Tgee80', 'H5qmIMX5JZ4']

['Tx-WtFiGgKw',
 'go-rbLUxtdA',
 '4fhLDOPiacE',
 'HR58Svuetw8',
 '0iTb19qV4Kw',
 'XSBaSS6HEOw',
 'NH6S8Tgee80',
 'H5qmIMX5JZ4']

In [11]:
import youtube_dl

def get_latest_video_url(channel_urls):
    video_urls = []
    for channel_url in channel_urls:
        with youtube_dl.YoutubeDL({'quiet': True}) as ydl:
            result = ydl.extract_info(channel_url, download=False)
            if 'entries' in result:
                # Sort videos by upload date
                latest_video = sorted(result['entries'], key=lambda x: x['upload_date'], reverse=True)[0]
                video_urls.append(latest_video['webpage_url'])
    return video_urls


In [12]:
import feedparser
from datetime import datetime, timedelta

# Function to get the latest video ID and upload date from a YouTube channel using RSS feed
def get_latest_video_id(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=2):
        # Extract video ID from the URL
        video_id = latest_video.link.split("=")[-1]
        # Return the video ID if the video is within one day old
        return video_id
    else:
        return None


def get_latest_video_name(channel_id):
    # Construct the RSS feed URL for the YouTube channel
    feed_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
    # Parse the feed
    feed = feedparser.parse(feed_url)
    # Get the first entry, which is the latest video
    latest_video = feed.entries[0]
    # Parse the published date
    published_date = datetime.strptime(latest_video.published, '%Y-%m-%dT%H:%M:%S+00:00')
    # Check if the video is less than a day old
    if datetime.utcnow() - published_date <= timedelta(days=2):
        # Extract video ID from the URL
        title= latest_video.title
        # Return the video ID if the video is within one day old
        return title
    else:
        return None

In [13]:
from youtube_transcript_api import YouTubeTranscriptApi


def get_youtube_captions(video_id):
    try:
        # Retrieve the available subtitles
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        current_minute = 0
        paragraph = []
        paragraphs = []
        
        # Process each subtitle line
        for item in transcript:
            # Calculate the minute mark of the current subtitle
            item_minute = int(item['start'] // 60)
            
            # Check if the current subtitle still falls within the current minute window
            if item_minute == current_minute:
                paragraph.append(item['text'])
            else:
                # Store the completed paragraph and start a new one
                paragraphs.append(' '.join(paragraph))
                paragraph = [item['text']]
                current_minute = item_minute
        
        # Add the last paragraph if any
        if paragraph:
            paragraphs.append(' '.join(paragraph))
        
        # Join all paragraphs with two newlines
        formatted_text = '\n\n'.join(paragraphs)
        
        # Optionally save the subtitles to a file
        with open(f'{video_id}_subtitles.txt', 'w', encoding='utf-8') as f:
            f.write(formatted_text)
        
        return formatted_text
    except Exception as e:
        return str(e)

# Assuming latest_video_ids is a list of video IDs
for video_id in latest_video_ids:
    captions = get_youtube_captions(video_id)
    print(captions)

guys we need to talk about this right now the best Market sector being semis is starting to disappoint in earnings and this could lead us into the almighty correction yes a movement down to potentially 4700 becomes more likely as we move back into what we call a negative gamma situation on top of all of this though crypto is also starting to fail often considered a risk on or risk-off asset it's falling below a very pivotal zone of 60,000 and as we know there's no shelves on the left hand side before 52k so could this be the crash on the horizon and more importantly what does that mean for us as stock Traders as we look at Tesla AMD and of course even gold not staying safe in this current market turmoil stocks Commodities and cryptos coming up and don't worry there's plenty of opportunities out there for all of us let's get into it well welcome back everybody to The Daily Show my name is Tom today we're discussing the latest macro the data the lead indicators and of course the hottest 

In [ ]:
'1R5gcXp8gNM'
'1w2cO2lSVkw'
'e9jF7GZzedg'



In [13]:
get_youtube_captions('sT-FyirmV_E')

"good evening folks Michael Silva here you're watching the stock market brief show this is where we navigate the financial markets through the lens of Technical and intermarket Analysis today we're going to be talking about a couple of areas where to be bullish and where to be bearish I'm going to give you why I'm thinking this and what to expect going into the remainder of this week we're going to start off right now looking at a chart of the S&P 500 we know that we've seen a lot of volatility to the downside coming from the highs to the lows roughly about almost 6% as far as a drop goes and you know this is the first 6% drop that we've seen since I mean for a while right we haven't seen really any pullback since October of the prior year and we came just under this 5,000 level and we shot back up this was actually something that we discussed as far as the call Wall goes where last time where we saw this kind of bare Market 4200 was the call wall or sorry the put wall and we cut throu

In [ ]:
"Analyze the content from the file I've provided, to answer: What is the short term forecast of the market, especially SPY and ETFs. Give me the price ranges, bullish cases or indicators, and bearish cases or indicators.'



In [ ]:
'Analyze the content from the file I've provided, to answer: What is the short term forecast of the market.'

In [ ]:
'Outline the bullish and bearish cases. The amount of bullet points should reflect the analyst's stance on whether they are bullish or bearish overall. If there are any specific symbols and price levels to look out for, state them.'

In [ ]:
"Give me a summary and investment advice based on this analysis."

In [ ]:
import openai

def get_chatgpt_response(prompt_file_path):
    # Load your API key from an environment variable or direct string (not recommended for production)
    openai.api_key = 'your-api-key-here'

    # Read the prompt from the file
    with open(prompt_file_path, 'r') as file:
        prompt = file.read()

    # Send the prompt to the ChatGPT model
    response = openai.ChatCompletion.create(
        model="gpt-4",  # or use "text-davinci-003" for GPT-3.5
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Return the response text
    return response.choices[0].message['content']

# Example usage
#prompt_path = 'path_to_your_prompt_file.txt'  # Change this to the path of your prompt file
#response_text = get_chatgpt_response(prompt_path)
#print("ChatGPT response:", response_text)